# Information capacity of Deep learning manifolds

* **Authors**:

  * author: steevelaquitaine@epfl.ch; laquitainesteeve@gmail.com
  * adapted from notebooks by SueYeon Chung, and Cory Stephenson
  * duration: 40 minutes


* **Learning outcomes**:

  * Know how to train a deep learning model for object recognition
  * Know how to measure its layers' neural manifold geometry (`dimensionality`, `radius`, `correlation`)
  * Know how to measure its `information capacity`.

* **System requirements**:

  * GPU: RunTime - change runtime type - Hardware accelerator: T4 GPU
  * RAM: 12.67GB

* **Constrains**

  * GPU is available for a short period

* **Readings**

  * Stephenson, C., Feather, J., Padhy, S., Elibol, O., Tang, H., McDermott, J., & Chung, S. (2019). Untangling in invariant speech recognition. Advances in neural information processing systems, 32.
  * https://github.com/steevelaquitaine/neural_manifolds_replicaMFT_Cajal/blob/master/examples/MFTMA_VGG16_example.ipynb

**Python prerequisites**:

  * installing and importing `libraries`
  * using `functions`
  * plotting with `matplotlib` library
  * machine learning with `scikit-learn` and `pytorch` libraries

## (34s) Setup

In [ ]:
!python --version # should output Python 3.10.13

In [ ]:
# download information geometry software
!git clone https://github.com/steevelaquitaine/neural_manifolds_replicaMFT_Cajal.git

# install conda
!pip install -q condacolab
import condacolab
condacolab.install()
!which conda

# use conda to install dependencies of the info geometry software
!conda env update --name base --file /content/neural_manifolds_replicaMFT_Cajal/env.yml
!exit
!conda init
!conda activate base

# move to the downloaded software directory
import os
os.chdir("/content/neural_manifolds_replicaMFT_Cajal/")

# install the software
!pip install -q -e .

# install time tracking software
!pip install -q ipython-autotime # time track colab notebook cells
%load_ext autotime

In [3]:
import numpy as np
np.random.seed(0)

# plotting software
import matplotlib.pyplot as plt

# dataset software
import torch
from torchvision import datasets, transforms, models

# information geometry software
from mftma.manifold_analysis_correlation import manifold_analysis_corr
from mftma.utils.make_manifold_data import make_manifold_data
from mftma.utils.activation_extractor import extractor
from mftma.utils.analyze_pytorch import analyze

time: 3.09 s (started: 2024-03-27 10:45:54 +00:00)


In [4]:
# define custom functions
def train(model, device, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        output = torch.nn.functional.log_softmax(output, dim=1)
        loss = torch.nn.functional.nll_loss(output, target)
        loss.backward()
        optimizer.step()

def test(model, device, test_loader, epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            output = torch.nn.functional.log_softmax(output, dim=1)
            test_loss += torch.nn.functional.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set epoch {}: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        epoch, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

time: 1.58 ms (started: 2024-03-27 10:46:04 +00:00)


## Train a Deep learning model of object recognition
* The model is VGG16

* The image dataset is CIFAR-100
  * 100 image classes
  * 50 images per class

In [6]:
# setup parameters
N_CLASSES = 100 # number of image classes in CIFAR-100
N_ITER = 20

examples_per_class = 50

time: 555 µs (started: 2024-03-27 10:46:45 +00:00)


In [7]:
# @title (21s) Create train, test datasets

mean = (0.5070751592371323, 0.48654887331495095, 0.4409178433670343)
std = (0.2673342858792401, 0.2564384629170883, 0.27615047132568404)

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

train_dataset = datasets.CIFAR100('../data', train=True, download=True,
                   transform=transform_train)
test_dataset = datasets.CIFAR100('../data', train=False, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize(mean, std)
                   ]))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1024, shuffle=True)

100%|██████████| 169001437/169001437 [00:01<00:00, 97701791.98it/s] 


Extracting ../data/cifar-100-python.tar.gz to ../data
Files already downloaded and verified
time: 5.9 s (started: 2024-03-27 10:46:50 +00:00)


In [ ]:
# @title (24m) Train and test the model
print("Using GPU: ", torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.vgg16(num_classes=N_CLASSES)
model = model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for i in range(N_ITER):#range(20):
    train(model, device, train_loader, optimizer)
    test(model, device, test_loader, i)

In [10]:
model = model.eval()

time: 1.75 ms (started: 2024-03-27 11:11:27 +00:00)


In [12]:
# @title (4s) Create the manifold dataset
# To create this, we have to specify the number of classes we want to sample,
# which in this case will just be the total number of samples in the dataset,
# so N_CLASSES=100. We also need to decide how many examples per class we
# want to use, and in this case we will use examples_per_class=50. Note that
# using large numbers of examples will result in a much longer runtime. We will
# also create the manifold data from the train dataset, and show the test
# dataset later.
data = make_manifold_data(train_dataset, N_CLASSES, examples_per_class, seed=0)
data = [d.to(device) for d in data]

time: 4.55 s (started: 2024-03-27 11:12:19 +00:00)


In [13]:
# @title (7s) Extract activations from the model
# Now we need to extract the activations at each layer of the model when the
# manifold data is given as an input. We can use the extractor given in
# mftma.utils.activation_extractor, which usually works, though depending on
# how the specific model is implemented, might miss some layers. If you do use
# it, make sure that all the layers you want to analyze are found! For this
# example, we will only look at the Conv2D and Linear layers of VGG16.
activations = extractor(model, data, layer_types=['Conv2d', 'Linear'])
list(activations.keys())

['layer_0_Input',
 'layer_1_Conv2d',
 'layer_3_Conv2d',
 'layer_6_Conv2d',
 'layer_8_Conv2d',
 'layer_11_Conv2d',
 'layer_13_Conv2d',
 'layer_15_Conv2d',
 'layer_18_Conv2d',
 'layer_20_Conv2d',
 'layer_22_Conv2d',
 'layer_25_Conv2d',
 'layer_27_Conv2d',
 'layer_29_Conv2d',
 'layer_33_Linear',
 'layer_36_Linear',
 'layer_39_Linear']

time: 6.81 s (started: 2024-03-27 11:12:23 +00:00)


In [14]:
# @title (7s) Describe first layer's activations
# a matrix of 50 examples per class x (unit receptive field size)
data = activations['layer_0_Input']
print(data[0].shape)

# X is a list of 100 arrays, one for each class
# each array is 50 instances x 3072 features (flattened filter = 3*32*32)
X = [d.reshape(d.shape[0], -1).T for d in data]

(50, 3, 32, 32)
time: 1.39 ms (started: 2024-03-27 11:12:37 +00:00)


In [15]:
# type(activations)
# remove some layers to fit in RAM memory
del activations['layer_1_Conv2d']
del activations['layer_3_Conv2d']
del activations['layer_6_Conv2d']
del activations['layer_8_Conv2d']
del activations['layer_11_Conv2d']
del activations['layer_13_Conv2d']
del activations['layer_15_Conv2d']
del activations['layer_18_Conv2d']
del activations['layer_20_Conv2d']
del activations['layer_22_Conv2d']
del activations['layer_25_Conv2d']
del activations['layer_27_Conv2d']
del activations['layer_29_Conv2d']
del activations['layer_33_Linear']
del activations['layer_36_Linear']
del activations['layer_39_Linear']

time: 7.68 ms (started: 2024-03-27 11:12:47 +00:00)


In [38]:
# number of image classes in layer 0
print(len(activations['layer_0_Input']), "image classes")

# description of layer 0 activations for the first image class
n_activation_features = activations['layer_0_Input'][0].shape[0]
n_image_per_class = activations['layer_0_Input'][0].shape[1]
print(n_activation_features,"activation features x", n_image_per_class, "image per class")

100 image classes
3072 activation features x 50 image per class
time: 845 µs (started: 2024-03-27 11:28:13 +00:00)


In [16]:
# @title (7s) Reduce feature dimensionality

# Now we're almost ready to run the analysis on the extracted activations.
# The final step is to convert them into the correct shape and pass them to
# manifold_analysis_corr. For example, the shape of the activations in
# layer_1_Conv2d is (50, 64, 32, 32), which we will flatten to (50, 65536) and
# transpose to the (65536, 50) format the analysis expects. This flattening may
# not always be appropriate as one might want to analyze each spatial location
# (or each timestep of a sequence model) independently.
# Additionally, the number of features here is quite a bit larger than needed,
# so we'll also do a random projection to 5000 dimensions to save time and
# memory usage. This step is optional and shouldn't change the geometry too
# much (see the Johnson–Lindenstrauss lemma) but it is useful to save on computation.

# takes 1 ms per layer

for layer, data, in activations.items():

    # X is a list of 100 array, one for each image class
    # each array is 50 sample instances x N features
    X = [d.reshape(d.shape[0], -1).T for d in data]

    # Get the number of features in the flattened data
    N = X[0].shape[0]

    # If N is greater than 5000, do the random projection to 5000 features
    if N > 5000:
        print("Projecting {}".format(layer))
        M = np.random.randn(5000, N)
        M /= np.sqrt(np.sum(M*M, axis=1, keepdims=True))
        X = [np.matmul(M, d) for d in X]
    activations[layer] = X

time: 1.05 ms (started: 2024-03-27 11:12:52 +00:00)


## (4m) Measure the manifold geometry

In [ ]:
# analyze the layer's class activations
a, r, d, correlation, K = manifold_analysis_corr(activations['layer_0_Input'], 0, 300, n_reps=1)

# compute the geometry metrics
capacity = 1/np.mean(1/a)
radius = np.mean(r)
dimension = np.mean(d)

In [45]:
print("capacity: {:4f}, radius {:4f}, dimension {:4f}, correlation {:4f}".format(capacity, radius, dimension, correlation))

capacity: 0.040322, radius 1.558389, dimension 34.696575, correlation 0.326515
time: 5.49 ms (started: 2024-03-27 11:41:24 +00:00)
